# Exploring pg_tables

In order to make a query plan, the query planner needs basic information like table names, column names, and the number of rows of each table or view. Without this information, the planner would not know how many rows to search to find a specific columns used in filter conditions. It would not know if it can use an index search.

This type of metadata is stored in the pg_tables schema. Specifically, the planner uses pg_class and pg_stats. Take a look to see what kind of information these objects contain.

```
SELECT * -- Index indicator column
FROM pg_class
WHERE relname = 'daily_aqi';
```

```
SELECT *
FROM pg_stats
WHERE tablename = 'daily_aqi'
AND attname = 'category';
```

# Basic EXPLAIN

Checking the query plan is an excellent window into the query function. Seeing the query plan steps helps you understand if you can restructure your query or add additional conditions to limit the rows and improve query speed. Comparing the relative costs between different query structures helps you see which plan is more efficient.

Here, you will be examining USA daily air quality data. First, familiarize yourself with running a query that will show the query plan.

```
EXPLAIN
SELECT * 
FROM daily_aqi;
```

# Index scans

Your friend lives in Hawaii where the volcanic smog is aggravating his asthma. You are helping him research new locations to live by using the Environmental Protection Agency's county level air quality data. You decide his new home should have good air quality at least 90% of the days.

You know there are a lot of counties in the USA, and the air quality table is large. Your first query (not shown) did a sequential scan and took a long time to run.

You decide to add an index to the column tracking the number of good air quality days. Examine the query plan and see the type of scan.

```
CREATE INDEX good_index 
ON annual_aqi(good);

EXPLAIN
SELECT state_name, county_name, aqi_year, good
FROM annual_aqi
WHERE good > 327 -- 90% of the year
AND aqi_year IN (2017,2018);
```

# EXPLAIN the WHERE

Running a query with EXPLAIN does not actually run the query. Thus, EXPLAIN provides estimates of the query plan. The cost, rows, and width are all estimates. Some query structures result in better estimates than other structures.

You will take another look at Hawaiian air quality, focusing on the days where multiple locations fell into that AQI category (good, moderate, etc.). You will then look at the impact of a partition on the query plans estimates. Does the partition change the planner's estimates?

```
SELECT COUNT(*)
FROM daily_aqi
WHERE state_code = 15 -- Hawaii state code
AND no_sites > 1;
```

```
EXPLAIN
SELECT *
FROM daily_aqi
WHERE state_code = 15 -- Hawaii state code
AND no_sites > 1;
```

```
EXPLAIN
SELECT *
FROM daily_aqi_partitioned
WHERE state_code = 15 -- partitioned on state code
AND no_sites > 1;
```

# EXPLAIN parameters

EXPLAIN gives a window into the query plan. It has some optional parameters that provide additional information. You want to try out these optional parameters, focusing on the VERBOSE and ANALYZE parameters.

VERBOSE allows you to see the available columns at each step of the plan. ANALYZE computes actual run times in milliseconds. Which parameter's output do you find most useful?

```
EXPLAIN VERBOSE
SELECT *
FROM country_demos;
```

```
EXPLAIN ANALYZE
SELECT *
FROM country_demos;
```

# Aggregating and sorting populations

ou are interested in planning your next vacation. You think it would be fun to visit a country that has changed a lot in recent years. You figure one way to look at change is to look at a change in population. So you focus on countries with big changes in population over the past 25 years.

You have annual population data from the World Bank covering each year from 1990 to 2017. Find the countries with the biggest population changes. Check your query plan to see how the aggregations impact the plan.

```
EXPLAIN ANALYZE
SELECT country
 , region
 , MAX(population) - MIN(population) as population_change
FROM country_pop 
GROUP BY country, region;
```

```
EXPLAIN ANALYZE
SELECT country
 , region
 , MAX(population) - MIN(population) as population_change
FROM country_pop 
GROUP BY country, region
ORDER BY population_change DESC;
```

# Joining in the query plan

China and India each have over 1 billion people and together contain over 30% of the world's people. You want to see if they are the countries with the biggest population growth.

Since they are both in Asia, you gather population data from the World Bank for every Asian country between 1990 to 2017. To simplify your analysis, you decide to compare two years of population data. You compare the recent (2017) population to the 1990 population to find a population growth metric.

Do India and China top the list? Are they the Asian countries with the most growth since the 1990s?

After writing your query, check the query plan to see how the join impacts the plan.

```
EXPLAIN ANALYZE
SELECT old.country
, old.region
, old.population_1990
, new.population_2017
, 100*((new.population_2017 - old.population_1990)/new.population_2017::float) as population_growth
FROM pop_1990 old
INNER JOIN pop_2017 new 
USING(country)
ORDER BY population_growth DESC
```

# Subqueries vs. CTEs

You have learned different ways of writing queries and how these choices impact performance. This exercise will again look at country demographics for the African countries with athletes competing in the Olympics. You will focus on the Summer Olympics.

Using pre-written queries, you will restructure the queries and assess the impact of the changes. You will start by seeing the difference in query plans when using subqueries compared to common table expressions (CTEs).

Run the execution plan for each step.

```
-- Subquery
EXPLAIN ANALYZE
SELECT city
, sex
, COUNT(DISTINCT athlete_id) as no_athletes
, AVG(age) as avg_age
FROM athletes_summ
WHERE country_code IN (SELECT olympic_cc FROM demographics WHERE gdp > 10000 and year = 2016)
AND year = 2016
GROUP BY city, sex;
```

Putting `WHERE` subquery inside CTE:

```
-- Note the initial step in the query plan

-- Common Table Expression (CTE)
EXPLAIN ANALYZE
WITH gdp AS -- From the demographics table
(
  SELECT olympic_cc FROM demographics WHERE gdp > 10000 and year = 2016
)
SELECT a.city, a.sex
  , COUNT(DISTINCT a.athlete_id) as no_athletes
  , AVG(a.age) as avg_age
FROM athletes_summ a
INNER JOIN gdp
  ON a.country_code = gdp.olympic_cc
WHERE a.year = 2016
GROUP BY a.city, a.sex;
```

# Why the difference?

The execution plan differs when using a subquery versus a common table expression (CTE) structure.

The following statements describe execution rules that are only true for CTEs. Choose the statement that describes a difference in the execution between a subquery and CTE.


- CTEs are only executed once per query, even if they are referenced more than one time in the parent query.
- CTEs create a temporary table.
- If a column with an index is referenced in a CTE, the index is also available outside of that CTE.

# Filtering impacts

You will revisit filtering while examining the African countries with athletes competing in the Summer Olympics. You will look at the average age by gender for the Rio de Janeiro and Beijing games.

Along the way, you will compare the query plans using numeric filters, text filters, and indexed filters.

```
-- Read the query plan with the text city filter
EXPLAIN ANALYZE
SELECT city, sex, COUNT(DISTINCT athlete_id), AVG(age) AS avg_age
FROM athletes_summ
WHERE city IN ('Rio de Janeiro','Beijing')
GROUP BY city, sex;
```

```
-- Find the execution time with a numeric year filter
EXPLAIN ANALYZE
SELECT city, sex, COUNT(DISTINCT athlete_id), AVG(age) AS avg_age
FROM athletes_summ
WHERE year IN (2016, 2008) -- Filter by year
GROUP BY city, sex;
```